# GMM para Speech features

## Importando libs

In [2]:
import numpy as np
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import random
from scipy.stats import multivariate_normal
import math
from numpy import linalg as LA

## Inicializando variáveis globais

In [3]:
Ng = 3
test_size = .2

## Lendo os dados MFCC do arquivo

In [8]:
target_frame_sanderson = pd.read_csv('data/feature_lib.csv')
target_frame_sanderson.fillna(0, inplace=True)
y = target_frame_sanderson.iloc[:, 1].values
X = target_frame_sanderson.iloc[:, 2:100].values
y, X

(array([ 1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,
         1.,  1.,  0.,  0.,  1.,  1.,  0.,  1.,  0.]),
 array([[ 16.06516658,  -6.167928  , -11.60263281, ...,   9.73565104,
          -6.01127765,  -0.39201773],
        [ 15.52645668,  -5.77143651,  -8.6210234 , ...,  25.84498242,
          -4.89838865,  11.15632049],
        [ 15.90670822,  -0.70229411, -11.62260717, ...,   1.67244094,
          -9.22338872,  -0.79974016],
        ..., 
        [ 16.03676337,  -3.01418019,  -2.49913762, ...,  14.47599253,
          -1.42811305,   2.77662196],
        [ 19.67184283, -14.94721738, -16.43764792, ...,   8.47230223,
          -9.10652143,   2.46218451],
        [ 16.03121297,  -3.3894565 , -12.06090029, ...,  14.06559439,
          -4.03672792,  -6.78197443]]))

## Separando treino e teste para validação cruzada

In [13]:
data, tar = X, y

def get_key(t):
    return t[1]

res = list(zip(data, tar))
res = sorted(res, key=get_key)
data, tar = list(zip(*res))
d0 = data[0:10]
d1 = data[10:20]
d2 = data[20:30]

t0 = tar[0:10]
t1 = tar[10:20]
t2 = tar[20:30]

dataset, target = np.array([d0, d1, d2]), np.array([t0, t1, t2])

n_interaction = [round(len(d) * test_size) for d in dataset]

data_tes = []
target_tes = []
data_train = []
target_train = []
for d in range(0, len(dataset)):
    for i in range(0, n_interaction[d]):
        rand_i = random.randint(0, len(dataset[d]) - 1)
        data_tes.append(dataset[d][rand_i])
        target_tes.append(target[d][rand_i])
        new_dataset = np.delete(dataset[d], rand_i, axis=0)
        new_target = np.delete(target[d], rand_i, axis=0)
    data_train.append(new_dataset)
    target_train.append(new_target)
X_train, y_train, X_test, y_test = data_train, target_train, data_tes, target_tes

## Dividindo o treinamento em classes